## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 3
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_3


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Las Vegas Raiders,3,2021,30.0,8.8,379.7,101.4,13.7,91.3,24.0,...,9.6,120.3,13.0,14.333333,14.000000,2.333333,1.586667,0.101695,2.000000,3.00
1,Arizona Cardinals,3,2021,34.3,9.9,322.3,113.4,12.9,110.0,21.7,...,10.3,140.7,25.0,10.333333,11.666667,1.333333,1.307692,0.033898,2.333333,1.75


In [10]:
X[X['offense_pass_TD_to_INT'] == np.inf]

,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
2,Minnesota Vikings,3,2021,29.0,7.7,297.0,118.4,10.4,128.0,26.0,...,13.3,119.3,29.333333,13.333333,10.333333,1.666667,1.266667,0.032258,2.666667,inf
10,Seattle Seahawks,3,2021,25.0,10.4,281.3,133.6,14.2,107.7,26.3,...,11.2,155.0,21.666667,9.666667,13.666667,1.333333,1.115385,0.095238,2.333333,inf
13,Denver Broncos,3,2021,25.3,8.7,260.0,116.4,11.3,127.3,8.7,...,9.9,59.3,19.333333,12.333333,12.000000,2.000000,2.142857,0.142857,1.333333,inf
17,New York Giants,3,2021,18.7,7.5,242.7,94.3,11.5,107.7,24.7,...,9.5,107.0,13.666667,12.666667,13.666667,1.666667,1.685714,0.149254,0.666667,inf


In [11]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [12]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
12,Los Angeles Rams,2021,week_3,0.329429,0.348833,0.508079,0.398877,0.396305
27,Cincinnati Bengals,2021,week_3,0.286888,0.302175,0.534814,0.406859,0.382684
7,Tampa Bay Buccaneers,2021,week_3,0.279407,0.280843,0.521823,0.339831,0.355476


In [13]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [14]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
12,Los Angeles Rams,2021,week_3,0.329429,0.348833,0.508079,0.398877,0.396305
27,Cincinnati Bengals,2021,week_3,0.286888,0.302175,0.534814,0.406859,0.382684
7,Tampa Bay Buccaneers,2021,week_3,0.279407,0.280843,0.521823,0.339831,0.355476
1,Arizona Cardinals,2021,week_3,0.311878,0.294928,0.392042,0.332020,0.332717
13,Denver Broncos,2021,week_3,0.292268,0.332915,0.322409,0.330112,0.319426
9,Carolina Panthers,2021,week_3,0.281551,0.309441,0.304322,0.380514,0.318957
0,Las Vegas Raiders,2021,week_3,0.286331,0.269565,0.298142,0.286359,0.285099
11,Buffalo Bills,2021,week_3,0.273334,0.239131,0.291716,0.297449,0.275407
30,New Orleans Saints,2021,week_3,0.241956,0.210902,0.278271,0.329357,0.265121
5,Kansas City Chiefs,2021,week_3,0.276510,0.235961,0.227545,0.256093,0.249027
